In [1]:
# !wget https://raw.githubusercontent.com/Vaseekaran-V/Datastorm-3/main/data/train_data_modified.csv?token=GHSAT0AAAAAABS4N64JYNY6EXMSZROSSTSOYSJQHMQ
# !wget https://raw.githubusercontent.com/Vaseekaran-V/Datastorm-3/main/data/validation_data.csv?token=GHSAT0AAAAAABS4N64JIDICIDNZNGXSE2HGYSIQRIQ
# !wget https://raw.githubusercontent.com/Vaseekaran-V/Datastorm-3/main/data/test_data.csv?token=GHSAT0AAAAAABS4N64J3BGH5UCQAEMW3NK2YSIQR7A

--2022-03-27 14:15:52--  https://raw.githubusercontent.com/Vaseekaran-V/Datastorm-3/main/data/train_data_modified.csv?token=GHSAT0AAAAAABS4N64JYNY6EXMSZROSSTSOYSJQHMQ
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 100306 (98K) [text/plain]
Saving to: ‘train_data_modified.csv?token=GHSAT0AAAAAABS4N64JYNY6EXMSZROSSTSOYSJQHMQ’

train_data_modified 100%[===================>]  97.96K  --.-KB/s    in 0.01s   

2022-03-27 14:15:52 (7.03 MB/s) - ‘train_data_modified.csv?token=GHSAT0AAAAAABS4N64JYNY6EXMSZROSSTSOYSJQHMQ’ saved [100306/100306]

--2022-03-27 14:15:53--  https://raw.githubusercontent.com/Vaseekaran-V/Datastorm-3/main/data/validation_data.csv?token=GHSAT0AAAAAABS4N64JIDICIDNZNGXSE2HGYSIQRIQ
Resolving raw.githubusercontent.com (raw.githubusercontent.com).

In [2]:
def week_id_valid(week):
  if week=="w1":    return 1
  elif week=="w2":    return 2
  elif week=="w3": return 3
  else: return 4

In [3]:
from sklearn.metrics import mean_squared_log_error, mean_absolute_percentage_error, mean_absolute_error
def calculate_errors(y_true, y_pred):
    return {'rmsle':mean_squared_log_error(y_true, y_pred, squared = False),
            'mape':mean_absolute_percentage_error(y_true, y_pred),
            'mae': mean_absolute_error(y_true, y_pred)
            }

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Read Data

In [9]:
train_data = pd.read_csv("/content/train_data_modified.csv?token=GHSAT0AAAAAABS4N64JYNY6EXMSZROSSTSOYSJQHMQ")
train_data

,CategoryCode,ItemCode,Week,id,WeeklySales
0,category_2,174436,w1,1,35
1,category_2,118033,w1,1,7
2,category_2,20824,w1,1,53
3,category_1,1061341,w1,1,18
4,category_1,76399,w1,1,21
...,...,...,...,...,...
3767,category_1,1044619,w4,20,45
3768,category_4,23200,w4,20,22
3769,category_1,1084498,w4,20,7
3770,category_4,872260,w4,20,12


In [7]:
valid_data = pd.read_csv('/content/validation_data.csv?token=GHSAT0AAAAAABS4N64JIDICIDNZNGXSE2HGYSIQRIQ')
valid_data['id'] = [week_id_valid(week)+20 for week in valid_data['Week']]
valid_data.sort_values(by='id', inplace = True)
valid_data

,CategoryCode,ItemCode,Week,WeeklySales,id
0,category_2,1044502,w1,11,21
142,category_1,48940,w1,22,21
269,category_1,35584,w1,15,21
138,category_2,59749,w1,33,21
137,category_2,132028,w1,112,21
...,...,...,...,...,...
139,category_2,118033,w4,12,24
140,category_2,1098502,w4,4,24
268,category_2,1010068,w4,19,24
146,category_2,211309,w4,23,24


In [14]:
test_data = pd.read_csv('/content/test_data.csv?token=GHSAT0AAAAAABS4N64J3BGH5UCQAEMW3NK2YSIQR7A')
test_data['id'] = [week_id_valid(week)+20 for week in test_data['Week']]
test_data.drop(['PredictedSales'],axis=1, inplace = True)

## Naive Forecast with Last Week Sales

In [10]:
train_data['Last_Week_Sales'] = train_data.groupby(['ItemCode','Week','CategoryCode'])['WeeklySales'].shift()
train_data

,CategoryCode,ItemCode,Week,id,WeeklySales,Last_Week_Sales
0,category_2,174436,w1,1,35,NaN
1,category_2,118033,w1,1,7,NaN
2,category_2,20824,w1,1,53,NaN
3,category_1,1061341,w1,1,18,NaN
4,category_1,76399,w1,1,21,NaN
...,...,...,...,...,...,...
3767,category_1,1044619,w4,20,45,29.0
3768,category_4,23200,w4,20,22,149.0
3769,category_1,1084498,w4,20,7,31.0
3770,category_4,872260,w4,20,12,36.0


In [11]:
train_data.dropna(inplace = True)

In [15]:
test_submission = pd.merge(test_data, train_data[['ItemCode','CategoryCode','Week','Last_Week_Sales']], how = "left", on=['ItemCode','CategoryCode','Week'])
test_submission.columns = ['CategoryCode'	, 'ItemCode'	,'Week',	'id'	, 'WeeklySales']
test_submission

,CategoryCode,ItemCode,Week,id,Last_Week_Sales
0,category_1,43738,w4,24,8.0
1,category_1,43738,w4,24,12.0
2,category_1,43738,w4,24,18.0
3,category_1,43738,w4,24,30.0
4,category_2,1006090,w1,21,2.0
...,...,...,...,...,...
1448,category_2,32245,w1,21,53.0
1449,category_2,1006090,w2,22,6.0
1450,category_2,1006090,w2,22,16.0
1451,category_2,1006090,w2,22,15.0


In [ ]:
test_submission['ID'] = [str(category)+'_'+str(item)+'_'+str(week) for category,item,week in zip(test_submission['CategoryCode'], test_submission['ItemCode'],test_submission['Week'])]
test_submission['ID']

# test_submission[['ID','WeeklySales']].to_csv('submission_naive_forecast_with_average.csv', index = False)

## Naive Forecast with Total Average Sales

In [ ]:
avg_sales = train_data.groupby(['ItemCode','CategoryCode'])['WeeklySales'].mean().to_frame('avg_sales')
avg_sales.reset_index(inplace = True)

In [ ]:
avg_sales

,ItemCode,CategoryCode,avg_sales
0,3418,category_1,63.400000
1,3427,category_1,42.500000
2,7666,category_3,83.650000
3,9925,category_2,11.900000
4,16936,category_3,45.150000
...,...,...,...
189,1101769,category_2,14.800000
190,1103056,category_2,16.333333
191,1105009,category_2,7.705882
192,1105018,category_2,7.687500


In [ ]:
test_data.sort_values('Week' ,inplace = True)
test_data

,CategoryCode,ItemCode,Week,PredictedSales
266,category_2,87559,w1,NaN
343,category_2,9925,w1,NaN
342,category_2,999403,w1,NaN
340,category_2,1103056,w1,NaN
336,category_2,838456,w1,NaN
...,...,...,...,...
364,category_2,87559,w4,NaN
325,category_3,37861,w4,NaN
34,category_1,1067119,w4,NaN
210,category_1,839374,w4,NaN


In [ ]:
test_submission = pd.merge(test_data, avg_sales[['ItemCode','CategoryCode','avg_sales']], how = "left", on=['ItemCode','CategoryCode'])
test_submission

,CategoryCode,ItemCode,Week,PredictedSales,avg_sales
0,category_2,87559,w1,NaN,24.300000
1,category_2,9925,w1,NaN,11.900000
2,category_2,999403,w1,NaN,15.150000
3,category_2,1103056,w1,NaN,16.333333
4,category_2,838456,w1,NaN,15.850000
...,...,...,...,...,...
372,category_2,87559,w4,NaN,24.300000
373,category_3,37861,w4,NaN,135.450000
374,category_1,1067119,w4,NaN,25.400000
375,category_1,839374,w4,NaN,40.200000


In [ ]:
test_submission['PredictedSales'] = test_submission['avg_sales']
test_submission.drop(['avg_sales'], axis = 1, inplace = True)
test_submission

,CategoryCode,ItemCode,Week,PredictedSales
0,category_2,87559,w1,24.300000
1,category_2,9925,w1,11.900000
2,category_2,999403,w1,15.150000
3,category_2,1103056,w1,16.333333
4,category_2,838456,w1,15.850000
...,...,...,...,...
372,category_2,87559,w4,24.300000
373,category_3,37861,w4,135.450000
374,category_1,1067119,w4,25.400000
375,category_1,839374,w4,40.200000


In [ ]:
test_submission.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 377 entries, 0 to 376
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   CategoryCode    377 non-null    object 
 1   ItemCode        377 non-null    int64  
 2   Week            377 non-null    object 
 3   PredictedSales  377 non-null    float64
 4   ID              377 non-null    int64  
 5   WeeklySales     377 non-null    float64
dtypes: float64(2), int64(2), object(2)
memory usage: 28.7+ KB


In [ ]:
test_submission['ID'] = range(0,len(test_submission))
test_submission['ID'] = [str(id) for id in test_submission['ID']]
test_submission['WeeklySales'] = test_submission['PredictedSales']

In [ ]:
test_submission.dtypes

CategoryCode       object
ItemCode            int64
Week               object
PredictedSales    float64
ID                 object
WeeklySales       float64
dtype: object

In [ ]:
test_submission['ID'] = [str(category)+'_'+str(item)+'_'+str(week) for category,item,week in zip(test_submission['CategoryCode'], test_submission['ItemCode'],test_submission['Week'])]
test_submission['ID']

test_submission[['ID','WeeklySales']].to_csv('submission_0_naive_forecast_with_average.csv', index = False)

0        category_2_87559_w1
1         category_2_9925_w1
2       category_2_999403_w1
3      category_2_1103056_w1
4       category_2_838456_w1
               ...          
372      category_2_87559_w4
373      category_3_37861_w4
374    category_1_1067119_w4
375     category_1_839374_w4
376      category_1_43738_w4
Name: ID, Length: 377, dtype: object

In [ ]:
test_submission[['ID','WeeklySales']].to_csv('submission_0_naive_forecast_with_average.csv', index = False)